In [ ]:
# https://medium.com/@iryna230520/dynamic-few-shot-prompting-overcoming-context-limit-for-chatgpt-text-classification-2f70c3bd86f9

In [1]:
USE_SAME_FEW_SHOTS=True

In [2]:
temp = 0.0
num_labels = 17
model_name="gpt-3.5-turbo-0301"
# model_name="gpt-3.5-turbo-instruct"
# model_name="gpt-4-0613"

In [3]:
output_dir = "./data/output/2023.11.21-1_GPT_few-shot-learning_all-17-sdg_1020-texts_GPT3.5_3-shots/"

final_xlsx_filename = output_dir+f"gpt_1000_texts_{num_labels}sdg_temperature_{temp}_model_name_{model_name}.xlsx"
print(final_xlsx_filename)

classification_report_filename = output_dir+f"gpt_1000_texts_{num_labels}sdg_temperature_{temp}_model_name_{model_name}_classification_report.xlsx"
print(classification_report_filename)

final_xlsx_filename_corrected = output_dir+f"gpt_1000_texts_{num_labels}sdg_temperature_{temp}_model_name_{model_name}_corrected.xlsx"
print(final_xlsx_filename_corrected)

./data/output/2023.11.21-1_GPT_few-shot-learning_all-17-sdg_1020-texts_GPT3.5_3-shots/gpt_1000_texts_17sdg_temperature_0.0_model_name_gpt-3.5-turbo-0301.xlsx
./data/output/2023.11.21-1_GPT_few-shot-learning_all-17-sdg_1020-texts_GPT3.5_3-shots/gpt_1000_texts_17sdg_temperature_0.0_model_name_gpt-3.5-turbo-0301_classification_report.xlsx
./data/output/2023.11.21-1_GPT_few-shot-learning_all-17-sdg_1020-texts_GPT3.5_3-shots/gpt_1000_texts_17sdg_temperature_0.0_model_name_gpt-3.5-turbo-0301_corrected.xlsx


In [4]:
# !pip install -q openai #--progress-bar off

In [5]:
# !pip install langchain

In [6]:
import os
from IPython.display import clear_output
os.environ["OPENAI_API_KEY"] = input("INSERT YOU OPENAI API KEY:")
clear_output()

In [7]:
import pandas as pd

import datetime
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

import os
import json
import ast
import re

from typing import Union
from IPython.display import display, HTML

from sklearn.metrics import classification_report

In [8]:
def perform_classification(text: str, chain: LLMChain) -> str:
    """Perform classification using LangChain model API.

    Parameters
    ----------
    text : str
        Input text.
    chain : LLMChain
        LangChain model.

    Returns
    -------
    str or None
        API Response
    """
    try:
        return chain.run(text=text)
    except Exception as e:
        print(e)
        return None
    
def convert_to_dict(row: str) -> dict:
    try:
        row = row.replace("false", "False")
        row = row.replace("true", "True")
        return ast.literal_eval(row)
    except Exception as e:
        print(e, row)
        return row

In [9]:
llm = ChatOpenAI(temperature=temp, model_name=model_name, max_tokens=500)

# Caricamento dati e preparazione prompt


In [10]:
# IN QUESTO ESPERIMENTO CONSIDERO LO STESSO DATASET UTILIZZATO IN 
# GPT_zero-shot_learning/2023.10.17-1_GPT_zero-shot-learning_all-17-sdg.ipynb
# IN MODO CHE I DATI SIANO GLI STESSI 

data = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_1020_texts_TEST.xlsx").drop(columns=['Unnamed: 0'])
# data.index = data.text_id
# data.text_id = data.index
# data = data.drop(columns=['Unnamed: 0', 'goal_label', 'motivation_for_the_answer', 'goal_label_int', 'sdg_probability'])
data

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0
1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0
1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0
1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0


In [11]:
data.shape

(1020, 7)

In [12]:
data.groupby('sdg').count()['text']

sdg
0     60
1     60
2     60
3     60
4     60
5     60
6     60
7     60
8     60
9     60
10    60
11    60
12    60
13    60
14    60
15    60
16    60
Name: text, dtype: int64

### Estraggo i dati per i few-shot

In [13]:
# TESTI ESTRATTI NEL FEW SHOT 2023.11.21-1_GPT_few-shot-learning_all-17-sdg_1020-texts_GPT3.5_3-shots

print("""
      text_id                   doi  \
3936     5614  10.2139/SSRN.1406042   
1456     2089  10.18356/ee52a573-en   
3974     5665  10.18356/3fe10a08-en   

                                                   text  sdg  labels_negative  \
3936  The modern definitions of the inchoate offence...   16              0.0   
1456  This refers to the manufacture, use and dispos...   12              3.0   
3974  As a result, both the trade and current accoun...    8              0.0   

      labels_positive  agreement  
3936              6.0     1.0000  
1456             29.0     0.8125  
3974              5.0     1.0000
""")


      text_id                   doi  3936     5614  10.2139/SSRN.1406042   
1456     2089  10.18356/ee52a573-en   
3974     5665  10.18356/3fe10a08-en   

                                                   text  sdg  labels_negative  3936  The modern definitions of the inchoate offence...   16              0.0   
1456  This refers to the manufacture, use and dispos...   12              3.0   
3974  As a result, both the trade and current accoun...    8              0.0   

      labels_positive  agreement  
3936              6.0     1.0000  
1456             29.0     0.8125  
3974              5.0     1.0000



In [14]:
if USE_SAME_FEW_SHOTS==True:
    data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx").drop(columns=['Unnamed: 0'])
    data_sel_4_few_shot_examples = data_sel_4_few_shot_examples[
        (data_sel_4_few_shot_examples.text_id==5614) |
        (data_sel_4_few_shot_examples.text_id==2089) |
        (data_sel_4_few_shot_examples.text_id==5665)
    ]
else:
    data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx").drop(columns=['Unnamed: 0']).sample(3)

data_sel_4_few_shot_examples

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement
1456,2089,10.18356/ee52a573-en,"This refers to the manufacture, use and dispos...",12,3.0,29.0,0.8125
3936,5614,10.2139/SSRN.1406042,The modern definitions of the inchoate offence...,16,0.0,6.0,1.0000
3974,5665,10.18356/3fe10a08-en,"As a result, both the trade and current accoun...",8,0.0,5.0,1.0000


In [15]:
few_shots = ""

for _, row in data_sel_4_few_shot_examples[['text', 'sdg']].iterrows():
    input_text = "Input text: \""+row['text']+"\"\n"
    output_text = "Output: \"SDG "+str(row['sdg'])+'\"\n\n'
    
    few_shots += input_text
    few_shots += output_text
    
print(few_shots)

Input text: "This refers to the manufacture, use and disposal of lifestyle goods such as electronics and clothing. Key sectors of high impact include food, housing and transport. In these sectors, wafer, energy and waste can be considered as crosscutting sectors affecting and affected by almost every lifestyle domain."
Output: "SDG 12"

Input text: "The modern definitions of the inchoate offences are well known in the criminal law since the Middle Ages both in European-Continent legal systems and in the English Common Law. They were mostly developed by the case-laws of the Star Chamber Court, which was abolished in 1640. The inchoate offences include three basic offences: Attempt, Conspiracy and Solicitation, but different legal systems defined some other offences as inchoate, such as the Aider and Abettor in Britain after the full validation of the Serious Crimes Act, 2007, c.27, s.44. In this paper it is argued, that there is a very intensive interaction between the fight against ter

In [16]:
# https://www.un.org/en/exhibits/page/sdgs-17-goals-transform-world#:~:text=GOAL%201%3A%20No%20Poverty%20GOAL,10%3A%20Reduced%20Inequalities%20GOAL%2011

template = """
Classify the following text between triple quotes according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015, choosing one (and only one) of the SDG labels based on it best representing the content of the entire text enclosed in triple quotes.

In the following lines, a text enclosed in triple quotes will be given to you.

The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

The following are 3 examples of expected classification given an input text:

"""+few_shots+"""


The text to be processed is:

```
{text}
```
"""

In [17]:
prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

In [18]:
chain = LLMChain(llm=llm, prompt=prompt)

In [54]:
# data_sel = data_sel_4_few_shot_examples
data_sel = data
# data_sel = data[data.sdg==0].sample(10)
# data_sel = data.sample(3)
# data_sel = data.iloc[356:]

data_sel

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0
1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0
1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0
1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0


In [32]:
# data_sel.iloc[:11]

In [31]:
# text_DEBUG = data_sel.iloc[10].text
# text_DEBUG

In [30]:
# perform_classification(text_DEBUG, chain)

In [36]:
# results = data_sel.apply(lambda x: perform_classication(x["text"], chain), axis=1)
import time

# cnt = 0
cnt = 355
results = []
for index, row in data_sel.iterrows():
    result = perform_classification(row["text"], chain)
    results.append((row["text_id"], result))
    cnt+=1
    time.sleep(1)
    print(cnt)
    

356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605


In [38]:
results

[(1510, 'SDG 5'),
 (747, 'SDG 4'),
 (630, 'SDG 5'),
 (5175, 'SDG 5'),
 (241, 'SDG 6'),
 (372, 'SDG 6'),
 (4479, 'SDG 6'),
 (1568, 'SDG 6'),
 (6194, 'SDG 6'),
 (650, 'SDG 6'),
 (4828, 'SDG 6'),
 (3495, 'SDG 6'),
 (2809, 'SDG 6'),
 (4972, 'SDG 6'),
 (3653, 'SDG 6'),
 (379, 'SDG 6'),
 (6287, 'SDG 6'),
 (6555, 'SDG 6'),
 (6626, 'SDG 6'),
 (3955, 'SDG 6'),
 (4253, 'SDG 6'),
 (5477, 'SDG 6'),
 (3247, 'SDG 6'),
 (598, 'SDG 6'),
 (4015, 'SDG 6'),
 (961, 'SDG 6'),
 (6717, 'SDG 6'),
 (1321, 'SDG 6'),
 (1347, 'SDG 6'),
 (827, 'SDG 6'),
 (4027, 'SDG 6'),
 (6358, 'SDG 14'),
 (3208, 'SDG 6'),
 (5966, 'SDG 6'),
 (4536, 'SDG 6'),
 (4687, 'SDG 3'),
 (5505, 'SDG 6'),
 (5928, 'SDG 6'),
 (5104, 'SDG 6'),
 (2755, 'SDG 6'),
 (3056, 'SDG 6'),
 (789, 'SDG 6'),
 (6331, 'SDG 6'),
 (1781, 'SDG 14'),
 (2291, 'SDG 6'),
 (2936, 'SDG 6'),
 (5851, 'SDG 6'),
 (2900, 'SDG 6'),
 (3427, 'SDG 6'),
 (3790, 'SDG 6'),
 (1506, 'SDG 6'),
 (4875, 'SDG 6'),
 (2949, 'SDG 6'),
 (2967, 'SDG 6'),
 (6342, 'SDG 6'),
 (983, 'SDG 6'),
 

In [40]:
# len(results)

664

In [42]:
# len(results_part1)

356

In [43]:
# results_tot = results_part1 + results

In [44]:
# len(results_tot)

1020

In [45]:
# results = results_tot

In [25]:
# results_part1 = results.copy()

In [26]:
# results_part1

[(6668, 'SDG 9'),
 (6099, 'SDG 0'),
 (6471, 'SDG 16'),
 (1786, 'SDG 4'),
 (2143, 'SDG 7'),
 (3377, 'SDG 0'),
 (5589, 'SDG 9'),
 (5431, 'SDG 8'),
 (768, 'SDG 7'),
 (2735, 'SDG 11'),
 (1306, 'SDG 8'),
 (3844, 'SDG 3'),
 (1524, 'SDG 3'),
 (4660, 'SDG 3'),
 (5077, 'SDG 0'),
 (995, 'SDG 8'),
 (2166, 'SDG 0'),
 (2512, 'SDG 16'),
 (2191, 'SDG 3'),
 (6073, 'SDG 0'),
 (1376, 'SDG 13'),
 (3889, 'SDG 3'),
 (6576, 'SDG 3'),
 (3912, 'SDG 3'),
 (1565, 'SDG 12'),
 (1033, 'SDG 16'),
 (5653, 'SDG 9'),
 (940, 'SDG 8'),
 (3613, 'SDG 0'),
 (1659, 'SDG 8'),
 (848, 'SDG 9'),
 (2157, 'SDG 15'),
 (3604, 'SDG 8'),
 (3940, 'SDG 3'),
 (6228, 'SDG 3'),
 (3148, 'SDG 0'),
 (2298, 'SDG 3'),
 (4226, 'SDG 8'),
 (2335, 'SDG 3'),
 (4832, 'SDG 3'),
 (6153, 'SDG 1'),
 (6772, 'SDG 8'),
 (6340, 'SDG 0'),
 (4505, 'SDG 3'),
 (1776, 'SDG 10'),
 (1529, 'SDG 9'),
 (1838, 'SDG 3'),
 (1427, 'SDG 15'),
 (2023, 'SDG 16'),
 (854, 'SDG 8'),
 (2541, 'SDG 3'),
 (6224, 'SDG 3'),
 (6243, 'SDG 16'),
 (4104, 'SDG 3'),
 (1497, 'SDG 9'),
 (20

In [46]:
len(results)

1020

In [47]:
results_processed = pd.DataFrame(results, columns=['text_id', 'result'])
results_processed['label_pred'] = results_processed['result'].str.extract(r'SDG (\d+)')
results_processed

,text_id,result,label_pred
0,6668,SDG 9,9
1,6099,SDG 0,0
2,6471,SDG 16,16
3,1786,SDG 4,4
4,2143,SDG 7,7
...,...,...,...
1015,4695,SDG 16,16
1016,6628,SDG 16,16
1017,4563,SDG 16,16
1018,6710,SDG 16,16


In [29]:
# results_processed.to_excel("gpt3_few_3_shot_tmp.xlsx")

In [ ]:
# results_processed = results.apply(convert_to_dict).apply(pd.Series).loc[:, ["motivation_for_the_answer", "detected_goals", "selected_goal"]]
# results_processed

In [ ]:
# results_processed2 = results.apply(convert_to_dict).apply(pd.Series).loc[:, ["goal_label", "motivation_for_the_answer", "sdg_probability"]]
# results_processed2

In [48]:
# results_processed = pd.DataFrame(results, columns=['text_id', 'result'])
# results_processed['label_pred'] = results_processed['result'].str.extract(r'SDG (\d+)')
# results_processed

In [55]:
# final = data_sel.merge(results_processed, left_index=True, right_index=True)
final = data_sel.merge(results_processed, left_on='text_id', right_on='text_id')
# final = data_sel.merge(results_processed2, left_index=True, right_index=True)
# final = final.drop(columns='Unnamed: 0')
final

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,result,label_pred
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,SDG 9,9
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,SDG 0,0
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,SDG 16,16
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,SDG 4,4
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN,SDG 7,7
...,...,...,...,...,...,...,...,...,...
1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,SDG 16,16
1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,SDG 16,16
1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,SDG 16,16
1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,SDG 16,16


In [56]:
final_xlsx_filename

'./data/output/2023.11.21-1_GPT_few-shot-learning_all-17-sdg_1020-texts_GPT3.5_3-shots/gpt_1000_texts_17sdg_temperature_0.0_model_name_gpt-3.5-turbo-0301.xlsx'

In [57]:
final.to_excel(final_xlsx_filename)

In [58]:
final_corrected = final

In [59]:
import numpy as np
y_true = final_corrected.sdg 
y_pred = final_corrected.label_pred
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       1.00      0.13      0.24        60
           1       0.71      0.78      0.75        60
           2       0.96      0.90      0.93        60
           3       0.64      0.98      0.78        60
           4       0.84      0.93      0.88        60
           5       0.90      0.87      0.88        60
           6       0.98      0.93      0.96        60
           7       0.87      0.88      0.88        60
           8       0.50      0.70      0.58        60
           9       0.71      0.77      0.74        60
          10       0.71      0.50      0.59        60
          11       0.91      0.80      0.85        60
          12       0.86      0.85      0.86        60
          13       0.79      0.88      0.83        60
          14       0.92      1.00      0.96        60
          15       0.93      0.88      0.91        60
          16       0.88      0.83      0.85        60
          17     

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [60]:
classification_report_filename

'./data/output/2023.11.21-1_GPT_few-shot-learning_all-17-sdg_1020-texts_GPT3.5_3-shots/gpt_1000_texts_17sdg_temperature_0.0_model_name_gpt-3.5-turbo-0301_classification_report.xlsx'

In [61]:
report=classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df.to_excel(classification_report_filename)

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
